#Classical kNN for Entanglement Classification

In this notebook, we one can replicate the simulation results of using kNN for entanglement classification that is provided as part of the work https://arxiv.org/abs/2003.09187. We use the kNN functionality in the scikit learn library with a custom metric, which is the fidelity. To use the custom metric, upload the two files accompanying this notebook, "cython_fidelity.pyx" and "setup.py" to the runtime and run the cell below.

In [ ]:
!python setup.py build_ext --inplace

Compiling cython_fidelity.pyx because it changed.
[1/1] Cythonizing cython_fidelity.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/cython_fidelity.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'cython_fidelity' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-OGiuun/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-OGiuun/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c cython_fidelity.c -o build/temp.linux-x86_64-3.7/cython_fidelity.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -

# Importing packages

In [ ]:
import numpy as np
import math
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from cython_fidelity import cython_fidelity

# Required functions
Here we define the functions that are necessary for our simulations

In [ ]:
#Return a single qubit rotation matrix

# 1. Theta: Angle of rotation

def RotMatCreate(Theta):
  
  return(np.matrix([[math.cos(Theta), (-1 * math.sin(Theta))], [math.sin(Theta), math.cos(Theta)]]))

In [ ]:
#Return matrix that rotates about the z axis in Bloch sphere

# 1. Theta: Angle of rotation

def ZRot(Theta):
  
  return(np.matrix([[math.cos(Theta/2) - (1j * math.sin(Theta/2)), 0],[0, math.cos(Theta/2) + (1j * math.sin(Theta/2))]]))

In [ ]:
# This function generates separable pure states.

# 1. QubitNo: Number of qubits

def GenSepState(QubitNo):
  
  FinalState = np.matrix([[1]])
    
  for i in range(QubitNo):
    FinalState = np.kron(FinalState,(2*np.random.random([2,1])-1 + 2j*np.random.random([2,1])-1))    
            
  FinalState = FinalState/np.linalg.norm(FinalState)
  return FinalState

In [ ]:
# This function generates entangled pure states.

# 1. QubitNo: Number of qubits.
# 2. IsMaxEnt: Enter 1 if the state should be maximally entangled.

def GenEntState(QubitNo, IsMaxEnt):
  
  if IsMaxEnt == 1:
        
    FinalState = np.zeros([2**QubitNo, 1])
    FinalState[0,0] = 1
    FinalState[-1,0] = 1
    
    RotMat = np.matrix([[1]])
    Thetas = (2*math.pi) * np.random.random(QubitNo)
    Phis = (math.pi) * np.random.random(QubitNo)
    
    Ry = [RotMatCreate(x/2) for x in Thetas]
    Rz = [ZRot(y) for y in Phis]
    MatY = np.matrix([[1]])
    MatZ = np.matrix([[1]])
    
    for i in range(QubitNo):
      MatY = np.kron(MatY, Ry[i])
      MatZ = np.kron(MatZ, Rz[i])

    FinalState = np.matmul(MatZ,np.matmul(MatY,FinalState))

  else:
        
    Re = (2 * np.random.random([2**QubitNo,1])) - 1 
    Im = (2 * np.random.random([2**QubitNo,1])) - 1
    FinalState = Re + 1j*Im
  
        
  FinalState = FinalState/np.linalg.norm(FinalState)
  return(FinalState)

In [ ]:
#Generate NSep separable states

# 1. NSep: Number of separable states to be generated. 
# 2. QubitNo: Number of qubits
# 3. Label: The label attached to the states for classification purposes.

def GenSepData(NSep, QubitNo, Label):
  SepData = np.zeros([NSep, 2**QubitNo]) + 1j*np.zeros([NSep, 2**QubitNo])
  for i in range(NSep):
    temp = GenSepState(QubitNo)
    SepData[i,:] = temp.T
  SepData = np.concatenate([SepData, Label*np.ones((NSep,1))], axis=1)
  return SepData

In [ ]:
#Generate 2 entangled states among 3 qubits.

# 1. QubitsToEntangle: 12 to entangle first 2 qubits
#                      23 to entangle last 2 qubits
#                      13 to entangle first and last qubits

def GenEntState2(QubitsToEntangle):
  if QubitsToEntangle == 12:
    return np.kron(GenEntState(2, 0), GenSepState(1)).T

  if QubitsToEntangle == 23:
    return np.kron(GenSepState(1), GenEntState(2, 0)).T

  if QubitsToEntangle == 13:
    temp = np.kron(GenSepState(1), GenEntState(2, 0))
    temp = np.reshape(np.array(temp.T), (2,2,2))
    temp = np.ndarray.transpose(temp, [1,0,2])
    temp = np.matrix(np.reshape(temp, (1,8)))
    return temp 

In [ ]:
# Generate and return NEnt number of entangled states. 
# These states would be fully entangled, that is, no bipartition of the qubits would result in two separable states.

# 1. NEnt: Number of states to be generated.
# 2. QubitNo: Number of qubits.
# 3. IsMaxEnt: Enter 1 if the states should be maximally entangled.
# 4. Label: The label attached to the states for classification purposes.

def GenEntData(NEnt, QubitNo, IsMaxEnt, Label):
  EntData = np.zeros([NEnt, 2**QubitNo]) + 1j*np.zeros([NEnt, 2**QubitNo]) 
    
  for i in range(NEnt):
    temp = GenEntState(QubitNo, IsMaxEnt)
    EntData[i,:] = temp.T
  EntData = np.concatenate([EntData, Label*np.ones((NEnt,1))], axis=1)
  return EntData

In [ ]:
# This function can generate a dataset of entangled states.

# 1. N: Number of states to be generated.
# 2. QubitsToEntangle: 12 to entangle first 2 qubits
#                      23 to entangle last 2 qubits
#                      13 to entangle first and last qubits
# 3. Label: A label for the entangled states. This label helps while providing the data to the KNN model. It can be any integer.

def GenMultiEntData(N, QubitsToEntangle, Label):
  Data = np.zeros([N, 8]) + 1j*np.zeros([N, 8])
  for i in range(N):
    Data[i,:] = GenEntState2(QubitsToEntangle)
  Data = np.concatenate([Data, Label*np.ones((N,1))], axis=1)
  return Data

# Testing
In this section, you may tweak the parameters and have a test run of the simulations. 


## Classical KNN

Here we have a test run of the classical kNN algorithm where the classification is done between separable vs entangled states with $100000$ states in each class.

In [ ]:
NSep = 100000
NEnt = 100000
n = 2

DataSep = GenSepData(NSep, n, 0)
Data12 = GenEntData(NEnt, n, 0, 1)
Data = np.concatenate([DataSep, Data12], axis = 0)

np.random.shuffle(Data)

DataX = Data[:,:-1]
DataY = Data[:,-1]
proportion = 400
DataSplitX = DataX
DataSplitY = np.real(DataY)
DataSplitX = np.concatenate([np.real(DataSplitX), np.imag(DataSplitX)], axis = 1)
XTrain, YTrain, XTest, YTest = DataSplitX[:-int((NSep+NEnt)/proportion)], DataSplitY[:-int((NSep+NEnt)/proportion)], DataSplitX[-int((NSep+NEnt)/proportion):], DataSplitY[-int((NSep+NEnt)/proportion):]
NeighComp = KNeighborsClassifier(n_neighbors=3, metric = cython_fidelity)
NeighComp.fit(XTrain, YTrain)
Ans = NeighComp.predict(XTest)
print("Accuracy = ",accuracy_score(YTest, Ans))
confusion_matrix(YTest, Ans)


Accuracy =  0.984


array([[250,   0],
       [  8, 242]])

#### 2 qubit with maximally entangled states

In [ ]:
NSep = 100000
NEnt = 100000
n = 2

DataSep = GenSepData(NSep, n, 0)
Data12 = GenEntData(NEnt, n, 1, 1)
Data = np.concatenate([DataSep, Data12], axis = 0)

np.random.shuffle(Data)

DataX = Data[:,:-1]
DataY = Data[:,-1]
proportion = 400
DataSplitX = DataX
DataSplitY = np.real(DataY)
DataSplitX = np.concatenate([np.real(DataSplitX), np.imag(DataSplitX)], axis = 1)
XTrain, YTrain, XTest, YTest = DataSplitX[:-int((NSep+NEnt)/proportion)], DataSplitY[:-int((NSep+NEnt)/proportion)], DataSplitX[-int((NSep+NEnt)/proportion):], DataSplitY[-int((NSep+NEnt)/proportion):]
NeighComp = KNeighborsClassifier(n_neighbors=3, metric = cython_fidelity)
NeighComp.fit(XTrain, YTrain)
Ans = NeighComp.predict(XTest)
print("Accuracy = ",accuracy_score(YTest, Ans))
confusion_matrix(YTest, Ans)


Accuracy =  1.0


array([[247,   0],
       [  0, 253]])

### 3 qubit

In [ ]:
N = 100000
DataSep = GenSepData(N, 3, 0)
Data12 = GenMultiEntData(N, 12, 12)
Data23 = GenMultiEntData(N, 23, 23)
Data13 = GenMultiEntData(N, 13, 13)
Data123 = GenEntData(N, 3, 0, 123)

Data = np.concatenate([DataSep, Data12, Data23, Data13, Data123], axis = 0)
np.random.shuffle(Data)

DataX = Data[:,:-1]
DataY = Data[:,-1]
proportion = 1000
DataSplitX = DataX
DataSplitY = np.real(DataY)
DataSplitX = np.concatenate([np.real(DataSplitX), np.imag(DataSplitX)], axis = 1)
XTrain, YTrain, XTest, YTest = DataSplitX[:-int((N * 5)/proportion)], DataSplitY[:-int((N * 5)/proportion)], DataSplitX[-int((N * 5)/proportion):], DataSplitY[-int((N * 5)/proportion):]
NeighComp = KNeighborsClassifier(n_neighbors=10, metric = cython_fidelity)
NeighComp.fit(XTrain, YTrain)
Ans = NeighComp.predict(XTest)
print("Accuracy = ",accuracy_score(YTest, Ans))
confusion_matrix(YTest, Ans)


Accuracy =  0.892


array([[103,   0,   0,   0,   0],
       [  6,  87,   0,   0,   0],
       [  6,   2,  89,   1,   0],
       [  8,   1,   1,  85,   0],
       [  0,   7,  11,  11,  82]])